In [1]:
import sys
sys.path.append('/home/npopkov/dll24')

import matplotlib.pyplot as plt
import torch
torch.backends.cudnn.benchmark = True

import numpy as np
import lightning as lt
torch.set_float32_matmul_precision('medium')
import soundfile as sf

from torch.utils.data import DataLoader,Dataset

import h5py
from tqdm.auto import tqdm




In [2]:
latents = np.load('latents.npy')
latent_dataset = torch.tensor(latents).float()
latent_dataloader = DataLoader(latent_dataset, batch_size=32, num_workers=30, drop_last=True)

In [3]:
# from audio_diffusion_pytorch import DiffusionModel, UNetV0, VDiffusion, VSampler

# model = DiffusionModel(
#     net_t=UNetV0, # The model type used for diffusion (U-Net V0 in this case)
#     in_channels=256, # U-Net: number of input/output (audio) channels
#     channels=[8, 32, 64, 128, 256, 512, 512], # U-Net: channels at each layer
#     factors=[1, 4, 4, 4, 2, 2, 2], # U-Net: downsampling and upsampling factors at each layer
#     items=[1, 2, 2, 2, 2, 2, 2], # U-Net: number of repeating items at each layer
#     attentions=[0, 0, 0, 0, 0, 1, 1], # U-Net: attention enabled/disabled at each layer
#     attention_heads=8, # U-Net: number of attention heads per attention item
#     attention_features=64, # U-Net: number of attention features per attention item
#     diffusion_t=VDiffusion, # The diffusion method used
#     sampler_t=VSampler, # The diffusion sampler used
# )

# class Lightningwrapper(lt.LightningModule):
#     def __init__(self, model):
#         super().__init__()
#         self.model = model
    
#     def forward(self, x):
#         return self.model(x)
    
#     def training_step(self, batch, batch_idx):
#         loss = self.model(batch)
#         self.log('train_loss', loss)
#         return loss
    
#     def configure_optimizers(self):
#         return torch.optim.Adam(self.model.parameters(), lr=3e-4)

# lt_model = Lightningwrapper(model)

In [4]:
from models.ddpm.CAUnet import CAUnet
from models.ddpm.DDPMBase import DDPMBase

batch_size = 32
n_epochs = 40
lr = 3e-4
n_steps, betaMin, betaMax = 200, 10**-6, 0.002

network = CAUnet(n_steps=n_steps,
                 c_in=256,
                 c_out=256,).to('cuda')

model = DDPMBase(network=network,
                betaMin=betaMin,
                betaMax=betaMax,
                n_steps=n_steps).to('cuda')

In [5]:

# Train model with audio waveforms
trainer = lt.Trainer(max_epochs=50)
trainer.fit(model, latent_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
2024-08-11 13:10:02.220487: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-11 13:10:02.333042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-11 13:10:02.381188: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-11 13:10:02.393428: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attem

Training: |          | 0/? [00:00<?, ?it/s]

torch.Size([32, 2048, 511]) torch.Size([32, 2048, 511])
torch.Size([32, 2048, 511]) torch.Size([32, 2048, 1022])


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 511 but got size 1022 for tensor number 1 in the list.